In [ ]:
# pip install -i https://pypi.tuna.tsinghua.edu.cn/simple agentscope --pre --user
# https://doc.agentscope.io/zh_CN/tutorial/103-example.html
# https://github.com/modelscope/agentscope/blob/main/README_ZH.md

In [ ]:
import os
import sys
from typing import Any
import json
import warnings; warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import torch as th
import agentscope
from agentscope import msghub
from agentscope.agents import (UserAgent, DialogAgent)
from agentscope.message import Msg
from agentscope.service import (ServiceToolkit, ServiceResponse)

In [ ]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")
devive_cnt = th.cuda.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(th.__version__)
print(th.version.cuda)
print(agentscope.__version__)